# Mapreduce

## Introduciton

We are going to be running mapreduce jobs on the wikipedia dataset.  The dataset is available (pre-chunked) on S3: `s3://dataincubator-course/mrdata/simple/`.  It may be downloaded with the `aws s3 sync` command or via HTTPS from `https://s3.amazonaws.com/dataincubator-course/mrdata/simple/part-000*`.

For development, you can even use a single chunk (eg. part-00026.xml.bz2). That is small enough that mrjob can process the chunk in a few seconds. Your development cycle should be:

1.  Get your job to work locally on one chunk.  This will greatly speed up your
development.  To run on local:
```bash
python job_file.py -r local data/wikipedia/simple/part-00026.xml.bz2 > /tmp/output.txt
```

### Note on Memory
There's a large difference between developing locally on one chunk and running your job on the entire dataset.  While you can get away with sloppy memory use locally, you really need to keep memory usage down if you hope to be able to complete the miniproject.  Remember, memory needs to be $O(1)$, not $O(n)$ in input.

### Multiple Mapreduces
You can combine multiple steps by overriding the [steps method](https://pythonhosted.org/mrjob/guides/writing-mrjobs.html#multi-step-jobs).  Usually your mapreduce might look like this
```python
from mrjob.job import MRJob

class SingleMRJob(MRJob):
    def mapper(self, key, value):
        pass

    def reducer(self, key, values):
        pass
```

`MRJob` automatically uses the `mapper` and `reducer` methods.  To specify multiple steps, you need to override the `steps` method:

```python
from mrjob.job import MRJob
from mrjob.step import MRStep

class MultipleMRJob(MRJob):
    def mapper1(self, key, value):
        pass

    def reducer1(self, key, values):
        pass
        
    def mapper2(self, key, value):
        pass

    def reducer2(self, key, values):
        pass
        
    def steps(self):
        return [
            MRStep(mapper=self.mapper1, reducer=self.reducer1),
            MRStep(mapper=self.mapper2, reducer=self.reducer2),
        ]
```






## Question 1: top100_words_simple_plain
Return a list of the top 100 words in an article text (in no particular order). You will need to write this as two map reduces:

1. The first job is similar to standard wordcount but with a few tweaks. The data provided for wikipedia is in `*.xml.bz2` format.  Mrjob will automatically decompress `bz2`.  We'll deal with the `xml` in the next question. For now, just treat it as text.  A few hints:
   - To split the words, use the regular expression "\w+".
   - Words are not case sensitive: i.e. "The" and "the" reference to the same word.  You can use `string.lower()` to get a single case-insenstive canonical version of the data.

2. The second job will take a collection of pairs `(word, count)` and filter for only the highest 100.

## Question 2: top100_words_simple_text
Notice that the words "page" and "text" make it into the top 100 words in the previous problem.  These are not common English words!  If you look at the xml formatting, you'll realize that these are xml tags.  You should parse the files so that tags like `<page></page>` should not be included in your total, nor should words outside of the tag `<text></text>`.

## Question 3: top100_words_simple_no_metadata

Finally, notice that 'www' and 'http' make it into the list of top 100 words in the previous problem.  These are also not common English words either!  These are clearly from the url in hyperlinks.  Looking at the format of [Wikipedia links](http://en.wikipedia.org/wiki/Help:Wiki_markup#Links_and_URLs) and [citations](http://en.wikipedia.org/wiki/Help:Wiki_markup#References_and_citing_sources), you'll notice that they tend to appear within single and double brackets and curly braces.


To eliminate the urls we use a package like the colorfully-named [mwparserfromhell](https://github.com/earwig/mwparserfromhell/), which has been provisioned on `mrjob` and supports the convenient helper function `strip_code()` (which is used by the reference solution).

## Question 4: wikipedia_entropy
The [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory) of a discrete random variable with probability mass function $p(x)$ is:

$$H(X) = - \sum p(x) \log_2 p(x)$$

You can think of the Shannon entropy as the number of bits needed to represent the random variable if it were optimally compressed.  It is also closely tied to the notion of entropy from physics.

You'll be estimating the Shannon entropy of different Simple English and Thai based off of their Wikipedias. Do this with n-grams of characters, by first calculating the entropy of a single n-gram and then dividing by n to get the per-character entropy. Use n-grams of size 1, 2, 3. 

The Thai Wikipedia is available at `s3://dataincubator-course/mrdata/thai/`.


Notes:
1. Characters are case sensitive.
1. Do not use the previous regex `\w+` to split --- depending on your system configuration, this may only match English characters, which would severely skew entropy estimates for Thai. Be careful about unicode.
1. Please treat all whitespace as the same character.  You can do this by
  `" ".join(text.split())`
1. For reference, the exact code we use to extract text is:

```
    wikicode = mwparserfromhell.parse(text)
    text = " ".join(" ".join(fragment.value.split())
                    for fragment in wikicode.filter_text())
```


## Question 5: link_stats_simple
Let's look at some summary statistics on the number of unique links on a page to other Wikipedia articles.  Return the number of articles (count), average number of links, standard deviation, and the 25%, median, and 75% quantiles.

1. Notice that the library `mwparserfromhell` supports the method `filter_wikilinks()`.
2. You will need to compute these statistics in a way that requires O(1) memory.  You should be able to compute the first few (i.e. non-quantile) statistics exactly by looking at the first few moments of a distribution. The quantile quantities can be accurately estimated by using reservoir sampling with a large reservoir.
3. If there are multiple links to the article have it only count for 1.  This keeps our results from becoming too skewed.
4. Don't forget that some (a surprisingly large number of) links have unicode! Make sure you treat them correctly.

## Question 6: link_stats_english
The same thing but for all of English Wikipedia. The data is also located on S3: `s3://dataincubator-course/mrdata/english/`.

**Note:**
Because of the size of the dataset, this job may take several hours to complete. It's advisable to run it overnight once you're reasonably sure it will work (due to testing the code on smaller inputs).

As a barometer, our reference solution takes around 5 hours to run.